# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

# FastSQL

A fun wrapper for SQL.

In [ ]:
#| export
from sqlalchemy import MetaData, Table, Column, Integer, String, ForeignKey, create_engine
from sqlalchemy import insert as _insert 
from sqlalchemy.orm import Session

from fastcore.utils import *
from dataclasses import dataclass, is_dataclass, asdict

__all__ = []

In [ ]:
#| export
class Database:
    def __init__(self, conn_str):
        self.conn_str = conn_str
        self.engine = create_engine(conn_str)
        self.metadata = MetaData()
        self.metadata.create_all(self.engine)

    def __repr__(self):
        return f"Database({self.conn_str})"

In [ ]:
Database("sqlite:///:memory:")

Database(sqlite:///:memory:)

In [ ]:
#| export
from typing import Any


class DBTable:
    def __init__(self, table: Table, database: Database):
        self._table = table
        self._database = database

    def __getitem__(self, name: str) -> Any:
        return self._table.c.__getattribute__(name)
    
    def __str__(self):
        return self._table.name
    
    def __repr__(self) -> str:
        return self._table.name

In [ ]:
#| export
type_map = {
    int: Integer,
    str: String,
    bool: Boolean
}
def create_column(name, typ, primary=False):
    return Column(name, type_map[typ], primary_key=primary)

In [ ]:
create_column("id", int, primary=True)

Column('id', Integer(), table=None, primary_key=True, nullable=False)

In [ ]:
create_column("name", str)

Column('name', String(), table=None)

In [ ]:
#| export
def _create_column_from_dataclass_field(name, field, primary=False):
    return create_column(name, field.type, primary)

In [ ]:
#| export
@patch
def create(self: Database, cls: dataclass, pk: str|None=None):
    pkcol = None
    cols = {k: v for k,v in cls.__dataclass_fields__.items()}
    # Set primary key, popping from cols
    if pk is not None: pkcol = _create_column_from_dataclass_field(pk, cols.pop(pk), primary=True)
    columns = [_create_column_from_dataclass_field(k, v) for k,v in cols.items()]
    # Insert primary key at the beginning
    if pkcol is not None: columns.insert(0, pkcol)
    # return Table(cls.__name__, self.metadata, *columns)
    return DBTable(Table(cls.__name__, self.metadata, *columns), self)

In [ ]:
db = Database("sqlite:///:memory:")
@dataclass
class User: name:str; pwd:str
users = db.create(User, pk='name')
print(users)
users

User


User

In [ ]:
@dataclass
class Todo: id:int; title:str; done:bool; name:str; details:str
todos = db.create(Todo, pk='id')
todos

Todo

In [ ]:
#| export
@patch
def insert(self: DBTable, **kwargs):
    with Session(self._database.engine) as session:
        stmt = _insert(self._table).values(**kwargs)
        result = session.execute(stmt)
        session.commit()
    return result

In [ ]:
todos.insert(title="Buy groceries", done=False, name="Alice", details="Milk, eggs, bread")

OperationalError: (sqlite3.OperationalError) no such table: Todo
[SQL: INSERT INTO "Todo" (title, done, name, details) VALUES (?, ?, ?, ?)]
[parameters: ('Buy groceries', 0, 'Alice', 'Milk, eggs, bread')]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()